In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


In [ ]:
!pip install pillow  # Ensure Pillow is installed for image handling

import os
import zipfile
from PIL import Image  # Import Pillow for image operations

# Assuming "archive (5).zip" is uploaded in the Colab root directory
zip_file_path = "archive (5).zip"
extraction_path = "fracture_images"  # Directory for extracted images

# Extract images from the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# Read and display a sample image
image_files = [f for f in os.listdir(extraction_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
if image_files:  # Check if any image files were found
    sample_image_path = os.path.join(extraction_path, image_files[0])
    img = Image.open(sample_image_path)
    img.show()  # Display the image
    print("ok")
else:
    print("No image files found in the extracted folder.")

# Further processing of images
# ... (Your code to load, manipulate, and use the extracted images)

No image files found in the extracted folder.


In [28]:
!pip install pillow  # Ensure Pillow is installed

import os
import zipfile
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Image Extraction ---
zip_file_path = "archive (5).zip"  # Assuming it's in the Colab root
extraction_path = "fracture_images"

# Extract images (if not already extracted)
if not os.path.exists(extraction_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)
    print("Images extracted to:", extraction_path)

# --- Data Loading ---
base_dir = extraction_path  # Update base_dir to the extraction path
img_size = (128, 128)

datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# --- Rest of your code (model building, training, etc.) ---
# ...

Found 6638 images belonging to 2 classes.
Found 1658 images belonging to 2 classes.


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),  # Increased units for better representation
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=3,  # Increased epochs to 25 (adjust as needed)
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)

Epoch 1/3
415/415 ━━━━━━━━━━━━━━━━━━━━ 289s 697ms/step - accuracy: 0.4914 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/3
415/415 ━━━━━━━━━━━━━━━━━━━━ 280s 675ms/step - accuracy: 0.5050 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/3
264/415 ━━━━━━━━━━━━━━━━━━━━ 1:39 660ms/step - accuracy: 0.4903 - loss: 0.6932

In [18]:
def predict_fracture(image_path, model, img_size=(128, 128)):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    prediction = model.predict(img_array)
    return 'Fractured' if prediction[0][0] > 0.5 else 'Non-Fractured'


In [27]:
print(predict_fracture('/content/broken-hand-xray.webp', model, img_size=(128, 128)))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Non-Fractured
